In [41]:
import numpy
import numpy as np
import math
import pandas as pd
from scipy.optimize import minimize
import itertools
from gt_design import *

In [42]:
data_efficiency = pd.read_csv(r'_outputs\_long_sim\output_data_efficiency.csv')
data_root_hub = pd.read_csv(r'_outputs\_long_sim\output_data_root_hub.csv')
data_off_design = pd.read_csv(r'_outputs\_long_sim\output_data_off_design.csv')
data_meanline = pd.read_csv(r'_outputs\_long_sim\output_data_meanline.csv')
data_meanline_losses = pd.read_csv(r'_outputs\_long_sim\output_data_meanline_losses.csv')

data_efficiency['delta_eta_optimize_normalized'] = (data_efficiency['delta_eta_optimize'] - data_efficiency['delta_eta_optimize'].min()) / (data_efficiency['delta_eta_optimize'].max() - data_efficiency['delta_eta_optimize'].min())
data_meanline_losses['N_rotor_normalized'] = (data_meanline_losses['N_rotor'] - data_meanline_losses['N_rotor'].min()) / (data_meanline_losses['N_rotor'].max() - data_meanline_losses['N_rotor'].min())
data_meanline['AN_squared_normalized'] = (data_meanline['AN_squared'] - data_meanline['AN_squared'].min()) / (data_meanline['AN_squared'].max() - data_meanline['AN_squared'].min())
data_efficiency['eta_opt'] = 1/data_efficiency['eta_final']
data_efficiency['eta_opt_normalized'] = (data_efficiency['eta_opt'] - data_efficiency['eta_opt'].min()) / (data_efficiency['eta_opt'].max() - data_efficiency['eta_opt'].min())

# OPTIMIZATION FUNCTION
#                                   EFFICIENCY                                      UPFRONT COST                                        OFF DESIGN                                                  MAINTENANCE
data_efficiency['func_optimize'] = 0.4 * (data_efficiency['eta_opt_normalized']) + 0.3 * (data_meanline_losses['N_rotor_normalized']) + 0.2 * (data_efficiency['delta_eta_optimize_normalized']) + 0.1 * data_meanline['AN_squared_normalized']

In [43]:
ind = data_efficiency['func_optimize'].idxmin()
data_efficiency.iloc[ind]

eta_tt                           90.233845
eta_final                         0.885367
eta_final_od                      0.869170
eta_final_od_new                  0.872020
eta_final_opt                     1.129475
delta_eta_optimize                0.013347
delta_eta_optimize_normalized     0.007192
eta_opt                           1.129475
eta_opt_normalized                0.109248
func_optimize                     0.220138
Name: 212999, dtype: float64

In [44]:
data_root_hub.iloc[ind]

reaction_hub        0.432210
h                   0.021463
r_tip               0.091891
U_tip             408.644692
r_hub               0.070428
r_tip_stator        0.095038
r_hub_stator        0.067281
h_vane_mean         0.027757
alpha_2_hub        68.794512
alpha_3_hub        28.251897
beta_2_hub         47.184652
beta_3_hub         57.201736
U_hub             313.196907
V_2_hub           307.609044
C_2_hub           577.978245
M_2_rel_hub         0.473185
M_2_hub             0.889085
reaction_hub.1      0.432210
alpha_2_tip        63.150363
alpha_3_tip        22.384291
beta_2_tip          1.189192
beta_3_tip         60.048478
U_tip.1           408.644692
V_2_tip           209.107782
C_2_tip           462.885959
M_2_rel_tip         0.317210
M_2_tip             0.702183
Name: 212999, dtype: float64

In [45]:
data_off_design.iloc[ind]

T_3_od                995.677810
rho_3_od                1.467235
P_3_od                419.276270
alpha_3_od             30.720649
beta_2_od              34.328232
flow_coeff_2_od         0.643609
incidence_2             7.295808
incidence_off           3.295808
v_2_od                253.157604
C_w_3_od              185.411297
C_a_3_od              312.012012
U_mean_od             324.828720
flow_coeff_3_od         0.960543
work_od_cw         212114.437944
work_od_vw         212114.437944
M_2_rel_od              0.384032
M_3_rel_od              0.968920
Name: 212999, dtype: float64

In [46]:
data_meanline.iloc[ind]

stage_loading            3.388889e+00
reaction_meanline        5.555556e-01
flow_coefficient_2       5.792483e-01
flow_coefficient_3       8.554217e-01
omega                    4.447062e+03
AN_squared               1.973788e+07
T_01                     1.245320e+03
T_1                      1.242085e+03
P_01                     1.182073e+03
P_1                      1.169840e+03
rho_1                    3.281655e+00
T_02                     1.249871e+03
T_2                      1.135607e+03
P_02                     1.041087e+03
P_02_rel                 7.713615e+02
P_2                      7.094773e+02
rho_2                    2.176851e+00
T_03                     1.053051e+03
T_3                      1.002508e+03
P_03                     5.245940e+02
P_03_rel                 7.593224e+02
P_3                      4.309006e+02
rho_3                    1.497639e+00
A_1                      1.736358e-02
A_2                      1.094488e-02
A_3                      1.094488e-02
M_1         

In [47]:
data_meanline_losses.iloc[ind]

c_true_stator            0.055513
c_a_stator               0.013430
stagger_angle_stator    51.022221
K_p_stator               0.025769
K_s_stator               0.094547
K_TET_stator             0.007875
K_stator                 0.128191
c_true_rotor             0.016510
c_a_rotor                0.014658
stagger_angle_rotor     27.401617
K_p_rotor                0.027228
K_s_rotor                0.075740
K_TET_rotor              0.006343
K_rotor                  0.109311
pitch_stator            14.855206
zweifel_stator           0.900000
N_stator                28.000000
pitch_rotor             43.741316
zweifel_rotor            0.950000
N_rotor                 42.000000
N_rotor_normalized       0.416667
Name: 212999, dtype: float64

In [50]:
stag_1 = aerodynamic_losses.profile_losses.figure_2_5(1.189192, 60.048478)
print(stag_1)

38.23725852273384


In [ ]:
c_true, c_a, N, o = aerodynamic_losses.trailing_edge_losses_rotor.required_vals(0.021463, 27.4016, 0.0811595, pitch_axial_chord_ratio,58.55)